# Neural Network with 3 hidden layers

### Load Modules

In [1]:
# Set the Random Seed
seed_value= 2022

import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import numpy  as np
import pandas  as pd
import datetime
import random
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.formula.api as smf
import tensorflow as tf
from keras.models     import Sequential, load_model
from keras.layers     import Activation, Dense, LeakyReLU
from keras.optimizers import Adam
from keras.callbacks  import EarlyStopping, ModelCheckpoint
from keras.regularizers import l1
from tensorflow.keras.losses import MeanSquaredError

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [3]:
def reset_seeds(seed_value=2022):
    os.environ['PYTHONHASHSEED']=str(seed_value)
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)
reset_seeds()

### Load Dataset - only traditional features

In [4]:
# Load monthly firm characteristics raw data
df = pd.read_parquet('C:/Users/rafae/Documents/HSG/Master Thesis/Data/Final/data07_model_input.parquet')
df = df.sort_values(by=['YM', 'permno'])
df = df.set_index(['year', 'YM', 'permno'])
df

gvkey  reprisk_id         prc         vol         mve_m  \
year YM      permno                                                             
2007 2007-01 10025    11903       37172   45.320000      8086.0  3.700557e+05   
             10026    12825       12684   39.689999      7613.0  7.653725e+05   
             10042    12139        4832    0.720000     26008.0  3.598898e+04   
             10078    12136        1719    6.130000  11333293.0  2.390900e+07   
             10104    12142        4413   16.430000   7234361.0  8.892640e+07   
...                     ...         ...         ...         ...           ...   
2021 2021-12 93304   184167       91339   36.750000    183303.0  1.695898e+06   
             93373   184323       74074    3.020000    528712.0  2.065325e+05   
             93374   184899       64442   74.510002    159495.0  8.587073e+06   
             93423    10567       22547   39.490002    254917.0  3.661156e+06   
             93436   184996       24952  936.719971   6361641.0  1.092218e+09   

                       absacc       acc    aeavol       age       agr  ...  \
year YM      permno                                                    ...   
2007 2007-01 10025   0.698728 -0.745547 -0.646819  0.457506 -0.979644  ...   
             10026   0.577608 -0.635623 -0.393384  0.457506  0.118575  ...   
             10042   0.990840 -0.989822 -0.894148  0.457506 -0.989822  ...   
             10078   0.654962 -0.711959 -0.128753  0.905344 -0.147074  ...   
             10104  -0.014758 -0.107379  0.780153  0.905344  0.770992  ...   
...                       ...       ...       ...       ...       ...  ...   
2021 2021-12 93304  -0.855522  0.706269 -0.871045 -0.500896  0.613134  ...   
             93373  -0.213134  0.143881  0.663284 -0.500896 -0.875821  ...   
             93374  -0.514030  0.423284 -0.223881 -0.500896  0.328955  ...   
             93423   0.242985 -0.303881  0.560597  0.143881 -0.584478  ...   
             93436   0.584478 -0.620299 -0.362388 -0.500896  0.856716  ...   

                     sic2_73  sic2_75  sic2_78  sic2_79  sic2_80  sic2_81  \
year YM      permno                                                         
2007 2007-01 10025         0        0        0        0        0        0   
             10026         0        0        0        0        0        0   
             10042         0        0        0        0        0        0   
             10078         0        0        0        0        0        0   
             10104         1        0        0        0        0        0   
...                      ...      ...      ...      ...      ...      ...   
2021 2021-12 93304         0        0        0        0        0        0   
             93373         0        0        0        0        0        0   
             93374         0        0        0        0        0        0   
             93423         0        0        0        1        0        0   
             93436         0        0        0        0        0        0   

                     sic2_82  sic2_83  sic2_87  sic2_99  
year YM      permno                                      
2007 2007-01 10025         0        0        0        0  
             10026         0        0        0        0  
             10042         0        0        0        0  
             10078         0        0        0        0  
             10104         0        0        0        0  
...                      ...      ...      ...      ...  
2021 2021-12 93304         0        0        0        0  
             93373         0        0        0        0  
             93374         0        0        0        0  
             93423         0        0        0        0  
             93436         0        0        0        0  

[338288 rows x 188 columns]

In [5]:
# Select only relevant columns for X and Y (exclude industry & reprisk rating dummy for now)
info_vars = ['YM', 'year', 'permno', 'gvkey', 'reprisk_id', 'prc', 'vol', 'mve_m']
X_vars = ['absacc', 'acc', 'aeavol', 'age', 'agr', 'baspread', 'beta', 'betasq', 'bm', 'bm_ia', 'cash', 'cashdebt',
          'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv', 'chmom', 'chpmia', 'chtx', 'cinvest',
          'convind', 'currat', 'depr', 'divi', 'divo', 'dolvol', 'dy', 'ear', 'egr', 'ep', 'gma', 'grcapx', 'grltnoa',
          'herf', 'hire', 'idiovol', 'ill', 'indmom', 'invest', 'lev', 'lgr', 'maxret', 'mom12m', 'mom1m', 'mom36m',
          'mom6m', 'ms', 'mve', 'mve_ia', 'nincr', 'operprof', 'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr',
          'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt', 'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc',
          'pricedelay', 'ps', 'quick', 'rd', 'rd_mve', 'rd_sale', 'realestate', 'retvol', 'roaq', 'roavol', 'roeq',
          'roic', 'rsup', 'salecash', 'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp', 'std_dolvol',
          'std_turn', 'stdacc', 'stdcf', 'tang', 'tb', 'turn', 'zerotrade']
sic2_vars = [col for col in df if col.startswith('sic2')]
reprisk_vars = ['country_sector_average', 'country_sector_average_01', 'current_rri', 'current_rri_01',
                'peak_rri', 'peak_rri_01', 'trend_rri', 'trend_rri_01']
reprisk_rating_vars = [col for col in df if col.startswith('reprisk_rating')]
Y_vars = ['ret', 'ret_wins', 'ret_ex']

# X
X = df[X_vars + sic2_vars]

# Y
Y = df[['ret_ex']]

X.shape, Y.shape

((338288, 162), (338288, 1))

### Hyperparameter Optimization: 4-Fold CV (12y/4 = 3y) and 3y Test

In [6]:
# Training (12y - 80%) and Test set (3y - 20%)
X_trai = X.loc['2007':'2018']
Y_trai = Y.loc['2007':'2018']

X_test = X.loc['2019':'2021']
Y_test = Y.loc['2019':'2021']

In [7]:
# 4-Fold cross validation (9y training and 3y validation)
K_FOLDs = 4
YEARS = list(X_trai.index.unique(level='year').astype('str')) 
TOT = len(YEARS)
TRA = int(TOT* (K_FOLDs-1) / K_FOLDs)
OFF = TOT - TRA

for FOLD in range(K_FOLDs):
    VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
    TRAI = [x for x in YEARS if x not in VALI]
    print(VALI, TRAI)    

['2007', '2008', '2009'] ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
['2010', '2011', '2012'] ['2007', '2008', '2009', '2013', '2014', '2015', '2016', '2017', '2018']
['2013', '2014', '2015'] ['2007', '2008', '2009', '2010', '2011', '2012', '2016', '2017', '2018']
['2016', '2017', '2018'] ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']


In [8]:
# Model
model_name = 'NN3_trad'

def create_NN3(l_rate=0.01, l1_pen=0.0001):
    model = Sequential()
    model.add(Dense(32,
                    activation='relu',
                    input_dim=X_trai.shape[1],
                    kernel_regularizer=l1(l1_pen)))
    model.add(Dense(16,
                    activation='relu',
                    kernel_regularizer=l1(l1_pen)))
    model.add(Dense(8,
                    activation='relu',
                    kernel_regularizer=l1(l1_pen)))
    model.add(Dense(1))
    model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=l_rate))
    return model

In [9]:
# Hyperparameter optimization
Y_val_preds = Y_trai.copy()
Y_test_preds = Y_test.copy()
results = []

earlyStopping  = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')

model_ix = 0
for l1_pen in [0.001, 0.0005, 0.0001, 0.00005, 0.00001]:
        for l_rate in [0.01, 0.005, 0.001]:
            for ensemble in list(range(0,10)):
                print("•", model_ix, ':',  end=' ')
                    
                for FOLD in range(K_FOLDs):
                    VALI = YEARS[(FOLD*OFF):((FOLD+1)*OFF)]
                    TRAI = [x for x in YEARS if x not in VALI]
                        
                    mcp_save = ModelCheckpoint(
                        '%s/%s_%d_%d.h5'%(model_name, model_name, model_ix, FOLD),
                        save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True
                    )

                    reset_seeds(ensemble*10000)
                        
                    NN_model = create_NN3(l_rate, l1_pen)
                        
                    history = NN_model.fit(
                        X_trai.loc[TRAI], Y_trai.loc[TRAI],
                        validation_data=(X_trai.loc[VALI], Y_trai.loc[VALI]),
                        epochs=200, verbose=0, shuffle=True, batch_size=2000,
                        callbacks=[earlyStopping, mcp_save]
                    )
                    tra_loss = np.min(history.history['loss'])
                    val_loss = np.min(history.history['val_loss'])

                    results.append({
                        'model_ix' :model_ix,
                        'l_rate'   :l_rate,
                        'l1_pen'   :l1_pen,
                        'fold'     :FOLD,
                        'tra_loss' :tra_loss,
                        'val_loss' :val_loss
                    })
                    
                    # Calculate validation predictions
                    best_model = create_NN3(l_rate, l1_pen)
                    best_model.load_weights(r'%s/%s_%d_%d.h5'%(model_name, model_name, model_ix, FOLD));
                    Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])
                    
                    # Calculate predictions for test data, if FOLD = 0
                    if FOLD==0:
                        Y_test_preds.loc[['2019','2020', '2021'], '%s_%d'%(model_name, model_ix)] = best_model.predict(X_test)

                model_ix += 1

• 0 : Epoch 12: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 12: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 9: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 10: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 1 : Epoch 12: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 20: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 9: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 13: early stopping
2066/2066 [==============================] - 5s 2ms/step
• 2 : Epoch 12: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 12: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 9: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 11: early stopping
2066/2066 [==============================] - 5s 2ms/step
•

2156/2156 [==============================] - 4s 2ms/step
Epoch 18: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 42 : Epoch 15: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 14: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 14: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 15: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 43 : Epoch 15: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 13: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 13: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 21: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 44 : Epoch 11: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 13: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 10: early stop

1929/1929 [==============================] - 4s 2ms/step
Epoch 46: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 55: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 52: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 84 : Epoch 32: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 45: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 54: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 46: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 85 : Epoch 78: early stopping
1929/1929 [==============================] - 4s 2ms/step
Epoch 70: early stopping
2193/2193 [==============================] - 4s 2ms/step
Epoch 74: early stopping
2156/2156 [==============================] - 4s 2ms/step
Epoch 75: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 86 : Epoch 41: early stop

C:\Users\rafae\AppData\Local\Temp\ipykernel_1696\2818418857.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])


1929/1929 [==============================] - 4s 2ms/step


...

Epoch 32: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 40: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 46: early stopping
2066/2066 [==============================] - 4s 2ms/step
• 149 : Epoch 50: early stopping
  16/1929 [..............................] - ETA: 6s 

C:\Users\rafae\AppData\Local\Temp\ipykernel_1696\2818418857.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_val_preds.loc[VALI, '%s_%d'%(model_name, model_ix)] = best_model.predict(X_trai.loc[VALI])


1929/1929 [==============================] - 4s 2ms/step


C:\Users\rafae\AppData\Local\Temp\ipykernel_1696\2818418857.py:52: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Y_test_preds.loc[['2019','2020', '2021'], '%s_%d'%(model_name, model_ix)] = best_model.predict(X_test)


Epoch 34: early stopping
2193/2193 [==============================] - 5s 2ms/step
Epoch 38: early stopping
2156/2156 [==============================] - 5s 2ms/step
Epoch 35: early stopping
2066/2066 [==============================] - 4s 2ms/step


In [10]:
# Save Y_val_preds
Y_val_preds.to_csv(r'%s/%s_val_preds.csv'%(model_name, model_name))
Y_val_preds

ret_ex  NN3_trad_0  NN3_trad_1  NN3_trad_2  NN3_trad_3  \
year YM      permno                                                             
2007 2007-01 10025  -0.031894    0.008004    0.008919    0.009752    0.008152   
             10026  -0.042317    0.008004    0.008919    0.009752    0.008152   
             10042  -0.125751    0.008004    0.008919    0.009752    0.008152   
             10078  -0.080607    0.008004    0.008919    0.009752    0.008152   
             10104  -0.046341    0.008004    0.008919    0.009752    0.008152   
...                       ...         ...         ...         ...         ...   
2018 2018-12 93420   0.086508    0.010904    0.002437    0.008059    0.004732   
             93422   0.466817    0.010904    0.002437    0.008059    0.004732   
             93423   0.105036    0.010904    0.002437    0.008059    0.004732   
             93429  -0.048712    0.010904    0.002437    0.008059    0.004732   
             93436  -0.079564    0.010904    0.002437    0.008059    0.004732   

                     NN3_trad_4  NN3_trad_5  NN3_trad_6  NN3_trad_7  \
year YM      permno                                                   
2007 2007-01 10025     0.008095    0.007825    0.009095    0.005856   
             10026     0.008095    0.007825    0.009095    0.005856   
             10042     0.008095    0.007825    0.009095    0.005856   
             10078     0.008095    0.007825    0.009095    0.005856   
             10104     0.008095    0.007825    0.009095    0.005856   
...                         ...         ...         ...         ...   
2018 2018-12 93420     0.011261    0.006232    0.008138    0.001793   
             93422     0.011261    0.006232    0.008138    0.001793   
             93423     0.011261    0.006232    0.008138    0.001793   
             93429     0.011261    0.006232    0.008138    0.001793   
             93436     0.011261    0.006232    0.008138    0.001793   

                     NN3_trad_8  ...  NN3_trad_140  NN3_trad_141  \
year YM      permno              ...                               
2007 2007-01 10025      0.00513  ...      0.013851      0.010490   
             10026      0.00513  ...      0.010050      0.010498   
             10042      0.00513  ...     -0.027764     -0.050386   
             10078      0.00513  ...      0.012945      0.010407   
             10104      0.00513  ...      0.011418      0.010533   
...                         ...  ...           ...           ...   
2018 2018-12 93420      0.00718  ...     -0.029808      0.001112   
             93422      0.00718  ...     -0.006863     -0.033720   
             93423      0.00718  ...      0.009498      0.006160   
             93429      0.00718  ...      0.016446      0.011639   
             93436      0.00718  ...      0.003162      0.002915   

                     NN3_trad_142  NN3_trad_143  NN3_trad_144  NN3_trad_145  \
year YM      permno                                                           
2007 2007-01 10025       0.012259      0.016665      0.010455      0.013330   
             10026       0.012259      0.010487      0.014244      0.013000   
             10042      -0.019268     -0.026286     -0.029779     -0.031946   
             10078       0.012259      0.011381      0.016308      0.012114   
             10104       0.012259      0.013091      0.015675      0.013872   
...                           ...           ...           ...           ...   
2018 2018-12 93420      -0.028944     -0.001328      0.019583     -0.023166   
             93422      -0.046797     -0.039256     -0.023008     -0.044846   
             93423       0.004473      0.006965      0.002910      0.012891   
             93429       0.015341      0.006965      0.014032      0.013194   
             93436       0.020902      0.007329      0.003668      0.013994   

                     NN3_trad_146  NN3_trad_147  NN3_trad_148  NN3_trad_149  
year YM      permno                                   

In [11]:
# Save Y_test_preds
Y_test_preds.to_csv(r'%s/%s_test_preds.csv'%(model_name, model_name))
Y_test_preds

ret_ex  NN3_trad_0  NN3_trad_1  NN3_trad_2  NN3_trad_3  \
year YM      permno                                                             
2019 2019-01 10026   0.004225    0.008004    0.008919    0.009752    0.008152   
             10104   0.036026    0.008004    0.008919    0.009752    0.008152   
             10107   0.075381    0.008004    0.008919    0.009752    0.008152   
             10138   0.072777    0.008004    0.008919    0.009752    0.008152   
             10145   0.076596    0.008004    0.008919    0.009752    0.008152   
...                       ...         ...         ...         ...         ...   
2021 2021-12 93304  -0.096386    0.008004    0.008919    0.009752    0.008152   
             93373  -0.019481    0.008004    0.008919    0.009752    0.008152   
             93374  -0.047552    0.008004    0.008919    0.009752    0.008152   
             93423  -0.072569    0.008004    0.008919    0.009752    0.008152   
             93436  -0.113609    0.008004    0.008919    0.009752    0.008152   

                     NN3_trad_4  NN3_trad_5  NN3_trad_6  NN3_trad_7  \
year YM      permno                                                   
2019 2019-01 10026     0.008095    0.007825    0.009095    0.005856   
             10104     0.008095    0.007825    0.009095    0.005856   
             10107     0.008095    0.007825    0.009095    0.005856   
             10138     0.008095    0.007825    0.009095    0.005856   
             10145     0.008095    0.007825    0.009095    0.005856   
...                         ...         ...         ...         ...   
2021 2021-12 93304     0.008095    0.007825    0.009095    0.005856   
             93373     0.008095    0.007825    0.009095    0.005856   
             93374     0.008095    0.007825    0.009095    0.005856   
             93423     0.008095    0.007825    0.009095    0.005856   
             93436     0.008095    0.007825    0.009095    0.005856   

                     NN3_trad_8  ...  NN3_trad_140  NN3_trad_141  \
year YM      permno              ...                               
2019 2019-01 10026      0.00513  ...      0.008643      0.010566   
             10104      0.00513  ...      0.007877      0.010578   
             10107      0.00513  ...      0.003714      0.010510   
             10138      0.00513  ...      0.007042      0.010468   
             10145      0.00513  ...      0.006492      0.010408   
...                         ...  ...           ...           ...   
2021 2021-12 93304      0.00513  ...      0.012077      0.010661   
             93373      0.00513  ...     -0.005942     -0.019556   
             93374      0.00513  ...      0.009092      0.010444   
             93423      0.00513  ...      0.013160      0.010636   
             93436      0.00513  ...     -0.001168      0.004194   

                     NN3_trad_142  NN3_trad_143  NN3_trad_144  NN3_trad_145  \
year YM      permno                                                           
2019 2019-01 10026       0.012259      0.009159      0.010160      0.012875   
             10104       0.012259      0.008483      0.015856      0.013279   
             10107       0.012259      0.010773      0.014458      0.012880   
             10138       0.011627      0.006776      0.009122      0.010100   
             10145       0.012259      0.006845      0.009955      0.007526   
...                           ...           ...           ...           ...   
2021 2021-12 93304       0.012259      0.012530      0.010848      0.010504   
             93373      -0.016302     -0.013161     -0.019003     -0.010910   
             93374       0.012259      0.009994      0.016466      0.013127   
             93423       0.012259      0.012746      0.016168      0.007342   
             93436      -0.013975      0.004441      0.012383      0.006642   

                     NN3_trad_146  NN3_trad_147  NN3_trad_148  NN3_trad_149  
year YM      permno                                   

In [12]:
# Result overview
table = pd.DataFrame(results)
table = table.groupby(['model_ix', 'l_rate', 'l1_pen']).mean().sort_values('val_loss')
table.to_csv(r'%s/%s_results.csv'%(model_name, model_name))

np.sqrt(table[['tra_loss', 'val_loss']].head(20)) * 100

,,,tra_loss,val_loss
model_ix,l_rate,l1_pen,,
115,0.001,0.00005,12.977783,12.945484
116,0.001,0.00005,12.980109,12.947809
110,0.001,0.00005,12.982489,12.950373
117,0.001,0.00005,12.983815,12.952406
112,0.001,0.00005,12.983659,12.952679
88,0.001,0.00010,12.989036,12.953050
85,0.001,0.00010,12.988798,12.954976
89,0.001,0.00010,12.989268,12.955285
87,0.001,0.00010,12.991380,12.958274


## Model Selection: Maximize SR on LS-Portfolio on validation data

In [13]:
# Model
model_name = 'NN3_trad'

# Get predictions
val = pd.read_csv(r'%s/%s_val_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])
test = pd.read_csv(r'%s/%s_test_preds.csv'%(model_name, model_name), index_col=['year', 'YM', 'permno'])

# Get overview table
table = pd.read_csv(r'%s/%s_results.csv'%(model_name, model_name)).sort_values(['model_ix'])

# Extract hyperparameters
list_l_rate = list(pd.unique(table['l_rate']))
list_l1_pen = list(pd.unique(table['l1_pen']))

In [14]:
# Loop over hyperparameter combinations to get ensemble predictions for each hyperparameter combination
val_results = []
hyper_comb = 0
for l_rate in list_l_rate:
    for l1_pen in list_l1_pen:
        # Get list of relevant model_ix
        list_model_ix = table[(table['l_rate'] == l_rate) & (table['l1_pen'] == l1_pen)]
        list_model_ix = list(list_model_ix['model_ix'])
        # Get list of column names
        list_col_names = []
        for ele in list_model_ix:
            list_col_names.append('%s_%d'%(model_name, ele))
        # Select relevant returns and average predictions
        val_ret = val[list_col_names]
        val_ret = val_ret.mean(axis=1)
        # Append ensemble return prediction to true returns
        comb = val[['ret_ex']].copy()
        comb['ret_pred'] = val_ret
        comb = comb.reset_index()
        # Sort the data by predicted returns and divide the data into quintiles
        comb['quintile'] = comb.groupby(['YM'])['ret_pred'].transform(lambda x: pd.qcut(x.rank(method='first'), 5, labels=np.arange(1,6)))
        # Calculate the mean return for each YM
        comb_mean = comb.groupby(['YM', 'quintile']).agg(ret_ex = ('ret_ex', 'mean'))
        # Add LS-Strategy
        comb_mean = comb_mean[['ret_ex']].unstack().add_prefix('Q')
        comb_mean.columns = comb_mean.columns.droplevel(0)
        comb_mean.columns.name = None
        comb_mean['LS'] = comb_mean['Q5'] - comb_mean['Q1']
        # Calculate the average return, standard deviation and Sharpe Ratio (annualized) per Quintile
        summary = pd.DataFrame()
        summary['mean'] = comb_mean.mean()
        summary['std'] = comb_mean.std()
        summary['SR'] = summary['mean'] / summary['std'] * np.sqrt(12)
        # Append results
        val_results.append({
            'hyper_comb':hyper_comb,
            'l_rate'    :l_rate,
            'l1_pen'    :l1_pen,
            'SR_Q1'     :summary.loc['Q1','SR'],
            'SR_Q5'     :summary.loc['Q5','SR'],
            'SR_LS'     :summary.loc['LS','SR'],
            'Mean_Q1'   :summary.loc['Q1','mean'],
            'Mean_Q5'   :summary.loc['Q5','mean'],
            'Mean_LS'   :summary.loc['LS','mean'],
        })
        hyper_comb += 1
        
# Save results
val_results = pd.DataFrame(val_results)
val_results.to_csv(r'results/{}_val.csv'.format(model_name))

In [15]:
# Select best hyperparemeters (max. SR_LS)
val_results = val = pd.read_csv(r'results/{}_val.csv'.format(model_name))
val_results = val_results.set_index(['l_rate', 'l1_pen']).sort_values(['SR_LS'], ascending=False)
l_rate_opt, l1_pen_opt = tuple([x for x in val_results.iloc[0].name[0:]])
print('Optimal l_rate:', l_rate_opt)
print('Optimal l1_pen:', l1_pen_opt)
val_results

Optimal l_rate: 0.001
Optimal l1_pen: 1e-05


Unnamed: 0  hyper_comb     SR_Q1     SR_Q5     SR_LS  \
l_rate l1_pen                                                          
0.001  0.00001          14          14  0.073488  0.393110  0.603447   
0.005  0.00001           9           9  0.040312  0.415203  0.596897   
0.010  0.00100           0           0  0.162740  0.531752  0.575295   
       0.00001           4           4  0.047440  0.458545  0.564957   
       0.00050           1           1  0.164962  0.508890  0.500430   
0.001  0.00005          13          13  0.128666  0.620777  0.394615   
0.005  0.00005           8           8  0.137656  0.564931  0.370869   
0.010  0.00010           2           2  0.337969  0.444874  0.259331   
0.005  0.00010           7           7  0.322487  0.438117  0.117206   
       0.00100           5           5  0.308877  0.354539  0.029507   
0.001  0.00010          12          12  0.390451  0.266074 -0.110900   
0.010  0.00005           3           3  0.298208  0.312984 -0.128704   
0.001  0.00050          11          11  0.425517  0.361857 -0.243399   
       0.00100          10          10  0.426279  0.295371 -0.270335   
0.005  0.00050           6           6  0.413575  0.294433 -0.290641   

                 Mean_Q1   Mean_Q5   Mean_LS  
l_rate l1_pen                                 
0.001  0.00001  0.001652  0.007038  0.005386  
0.005  0.00001  0.000948  0.007105  0.006157  
0.010  0.00100  0.003271  0.008005  0.004735  
       0.00001  0.001157  0.007579  0.006422  
       0.00050  0.003308  0.008244  0.004936  
0.001  0.00005  0.003111  0.008611  0.005500  
0.005  0.00005  0.003332  0.008258  0.004927  
0.010  0.00010  0.005791  0.007593  0.001802  
0.005  0.00010  0.006256  0.007154  0.000897  
       0.00100  0.005797  0.006078  0.000281  
0.001  0.00010  0.006106  0.005320 -0.000786  
0.010  0.00005  0.006591  0.005092 -0.001500  
0.001  0.00050  0.007345  0.006271 -0.001073  
       0.00100  0.006974  0.005469 -0.001505  
0.005  0.00050  0.007070  0.005376 -0.001694

## Out-of-Sample: Get Predictions of model with best hyperparameter combination

In [16]:
# Get list of model_ix's with best hyperparameter combinations and make ensemble predictions
list_model_ix = table[(table['l_rate'] == l_rate_opt) & (table['l1_pen'] == l1_pen_opt)]
list_model_ix = list(list_model_ix['model_ix'])

# Get list of column names
list_col_names = []
for ele in list_model_ix:
    list_col_names.append('%s_%d'%(model_name, ele))
    
# Select relevant returns and average predictions
test_ret = test[list_col_names].copy()
test_ret['y_pred'] = test_ret.mean(axis=1)

# Save predictions
test_ret = test_ret.reset_index()
test_ret = test_ret[['YM', 'permno', 'y_pred']]
test_ret.to_csv(r'results/{}_predictions.csv'.format(model_name), index=False)